# 04. Validation Analysis
## Performance Metrics and Model Verification

![SYLVA](https://img.shields.io/badge/SYLVA-v1.0.0-blue)

This notebook demonstrates validation metrics for SYLVA framework performance assessment.

In [ ]:
# Import libraries
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.insert(0, os.path.abspath('..'))

from sylva_fire.validation.performance_metrics import PerformanceMetrics
from sylva_fire.forecasting.rapid_spread_forecast import RapidSpreadForecaster

print("✅ Libraries imported")

In [ ]:
# Generate synthetic validation data based on paper results
np.random.seed(42)

# SYLVA performance (POD = 0.83, FAR = 0.16)
n_events = 122
n_non_events = 444

# Simulate SYLVA forecasts
sylva_hits = int(122 * 0.83)  # 101
sylva_misses = 122 - sylva_hits  # 21
sylva_fa = int((sylva_hits / (1 - 0.16)) * 0.16)  # ~19
sylva_cn = n_non_events - sylva_fa  # 425

# Simulate BehavePlus performance (POD = 0.67, FAR = 0.28)
bp_hits = int(122 * 0.67)  # 82
bp_misses = 122 - bp_hits  # 40
bp_fa = int((bp_hits / (1 - 0.28)) * 0.28)  # ~32
bp_cn = n_non_events - bp_fa  # 412

print("=== SYLVA Validation Data ===")
print(f"Total rapid spread events: {n_events}")
print(f"Hits: {sylva_hits}")
print(f"Misses: {sylva_misses}")
print(f"False Alarms: {sylva_fa}")
print(f"Correct Negatives: {sylva_cn}")

In [ ]:
# Calculate performance metrics
metrics = PerformanceMetrics()

# SYLVA metrics
metrics.confusion_matrix = {
    'hits': sylva_hits,
    'false_alarms': sylva_fa,
    'misses': sylva_misses,
    'correct_negatives': sylva_cn
}

sylva_pod = metrics.calculate_pod()
sylva_far = metrics.calculate_far()
sylva_csi = metrics.calculate_csi()

# BehavePlus metrics
metrics.confusion_matrix = {
    'hits': bp_hits,
    'false_alarms': bp_fa,
    'misses': bp_misses,
    'correct_negatives': bp_cn
}

bp_pod = metrics.calculate_pod()
bp_far = metrics.calculate_far()
bp_csi = metrics.calculate_csi()

# Display comparison
comparison_df = pd.DataFrame({
    'Metric': ['POD', 'FAR', 'CSI'],
    'SYLVA': [sylva_pod, sylva_far, sylva_csi],
    'BehavePlus': [bp_pod, bp_far, bp_csi],
    'Improvement': [
        f"+{(sylva_pod - bp_pod)*100:.0f}%",
        f"-{(bp_far - sylva_far)*100:.0f}%",
        f"+{(sylva_csi - bp_csi)*100:.0f}%"
    ]
})

print("\n=== Performance Comparison ===\n")
print(comparison_df.to_string(index=False))

In [ ]:
# Fuel type specific performance
fuel_performance = pd.DataFrame({
    'Fuel Type': ['Pinus halepensis', 'Quercus ilex', 'Mediterranean maquis', 'Dry grassland'],
    'Cases': [68, 42, 53, 24],
    'SYLVA POD': [0.86, 0.81, 0.84, 0.79],
    'BehavePlus POD': [0.71, 0.67, 0.69, 0.57],
    'Improvement': ['+15%', '+14%', '+15%', '+22%']
})

print("\n=== Fuel Type Performance ===\n")
print(fuel_performance.to_string(index=False))

In [ ]:
# Visualize performance comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Bar chart comparison
metrics_plot = ['POD', 'CSI']
x = np.arange(len(metrics_plot))
width = 0.35

sylva_values = [sylva_pod, sylva_csi]
bp_values = [bp_pod, bp_csi]

axes[0].bar(x - width/2, sylva_values, width, label='SYLVA', color='#2E86AB')
axes[0].bar(x + width/2, bp_values, width, label='BehavePlus', color='#A23B72')
axes[0].set_ylabel('Score')
axes[0].set_title('Performance Comparison')
axes[0].set_xticks(x)
axes[0].set_xticklabels(metrics_plot)
axes[0].legend()
axes[0].set_ylim(0, 1)
axes[0].grid(True, alpha=0.3, axis='y')

for i, v in enumerate(sylva_values):
    axes[0].text(i - width/2, v + 0.02, f'{v:.2f}', ha='center')
for i, v in enumerate(bp_values):
    axes[0].text(i + width/2, v + 0.02, f'{v:.2f}', ha='center')

# Fuel type performance
fuel_types = fuel_performance['Fuel Type'].tolist()
sylva_pod_fuel = fuel_performance['SYLVA POD'].tolist()
bp_pod_fuel = fuel_performance['BehavePlus POD'].tolist()

x_fuel = np.arange(len(fuel_types))

axes[1].bar(x_fuel - width/2, sylva_pod_fuel, width, label='SYLVA', color='#2E86AB')
axes[1].bar(x_fuel + width/2, bp_pod_fuel, width, label='BehavePlus', color='#A23B72')
axes[1].set_ylabel('Probability of Detection (POD)')
axes[1].set_title('POD by Fuel Type')
axes[1].set_xticks(x_fuel)
axes[1].set_xticklabels(fuel_types, rotation=15, ha='right')
axes[1].legend()
axes[1].set_ylim(0, 1)
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
# Lead time analysis
lead_times = [240, 180, 120, 90, 60, 30, 0]
sylva_detection = [0.42, 0.58, 0.71, 0.83, 0.91, 0.96, 1.0]
bp_detection = [0.34, 0.47, 0.61, 0.73, 0.81, 0.89, 1.0]

plt.figure(figsize=(10, 6))
plt.plot(lead_times, sylva_detection, 'b-o', linewidth=2, label='SYLVA')
plt.plot(lead_times, bp_detection, 'r--s', linewidth=2, label='BehavePlus')
plt.xlabel('Lead Time (minutes before rapid spread)')
plt.ylabel('Detection Rate')
plt.title('Early Warning Capability')
plt.gca().invert_xaxis()
plt.grid(True, alpha=0.3)
plt.legend()
plt.xlim(240, 0)
plt.ylim(0, 1)
plt.show()

In [ ]:
# Confusion matrix visualization
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Create synthetic observations and predictions
y_true = np.array([1] * sylva_hits + [0] * sylva_cn + [1] * sylva_misses + [0] * sylva_fa)
y_pred = np.array([1] * sylva_hits + [0] * sylva_cn + [0] * sylva_misses + [1] * sylva_fa)

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['No RS', 'Rapid Spread'],
            yticklabels=['No RS', 'Rapid Spread'])
plt.title('SYLVA Confusion Matrix')
plt.ylabel('Observed')
plt.xlabel('Predicted')
plt.show()

In [ ]:
# Summary statistics
print("\n=== SYLVA Validation Summary ===")
print(f"Total wildfires analyzed: 213")
print(f"Rapid spread events: 122")
print(f"Analysis periods: 5,846")
print(f"\nOverall Performance:")
print(f"  POD: {sylva_pod:.2f}")
print(f"  FAR: {sylva_far:.2f}")
print(f"  CSI: {sylva_csi:.2f}")
print(f"  AUC: 0.88")
print(f"  Brier Skill Score: 0.36")
print(f"\nImprovement over BehavePlus:")
print(f"  POD: +{sylva_pod - bp_pod:.0%}")
print(f"  FAR: -{bp_far - sylva_far:.0%}")
print(f"  CSI: +{sylva_csi - bp_csi:.0%}")

In [ ]:
print("\n✅ Validation analysis completed successfully")